In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import re
from time import sleep
from random import random
from time import time
import glob

In [ ]:
#Save Credentials
with open('SpotifyID.txt', 'r') as f:
    client_id = f.read().replace('\n','')
with open('SpotifySecret.txt', 'r') as f:
    client_secret = f.read().replace('\n','')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [ ]:
def query(data_cleaned,file_loc):
    count=0
    uris, dates, popularities, duration_ms=[],[],[],[]

    for i in range(0,len(data_cleaned)):  
        #query artist and song
        artist=str(data_cleaned.loc[i,"song_artist"])
        title=str(data_cleaned.loc[i,"song_name"])

        #some songs have many artists associated
        #however spotify only lists few, so break the string into parts 
        #and search till match is found
        artist.replace(","," ")
        artist_string = [artist] + artist.strip().split()
        for artist in artist_string:
            if len(artist)>2:
                search_query = str(title) + ' ' + str(artist)
                result = sp.search(search_query)
                if result["tracks"]["items"]!=[]:
                    break
        
        if result["tracks"]["items"]==[]:
            uris.append("error")
            dates.append("error")
            popularities.append("error")
            duration_ms.append("error")   
            print("error",count)
            count+=1
        
        else:
            print(i,result["tracks"]["items"][0]["uri"],artist)
            uri=result["tracks"]["items"][0]["uri"]
            date= result["tracks"]["items"][0]["album"]["release_date"]
            duration = result["tracks"]["items"][0]["duration_ms"]
            popularity = result["tracks"]["items"][0]["popularity"]  

            uris.append(uri)
            dates.append(date)
            popularities.append(popularity)
            duration_ms.append(duration)

    return uris, dates, popularities, duration_ms

In [ ]:
def create_csv(uris, dates, popularities, duration_ms, data_cleaned, file_loc):
    data_cleaned["uris"] = uris
    data_cleaned["duration_ms"] = duration_ms
    data_cleaned["popularity"] = popularities
    data_cleaned["date"] = dates
    data_cleaned.to_csv(file_loc)

In [ ]:
#read in songs from main df
datasets = glob.glob("./datasets/*.csv")
for file_loc in datasets:
    data_cleaned = pd.read_csv(file_loc)
    for column in data_cleaned.columns:
        data_cleaned[column] = data_cleaned[column].str.strip()
        data_cleaned = data_cleaned.dropna(subset =["song_name"]).reset_index(drop =True)
    file_loc = file_loc.replace("datasets","final_datasets")
    
    uris, dates, popularities, duration_ms = query(data_cleaned,file_loc)
    
    create_csv(uris, dates, popularities, duration_ms, data_cleaned, file_loc)
